In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from datetime import datetime, timedelta
from tqdm import tqdm

from matplotlib.dates import DateFormatter
import warnings
warnings.filterwarnings("ignore")

# Data
## Financials

In [ ]:
financial = pd.read_csv('data/train_files/financials.csv')

In [ ]:
financial.head(2)

In [ ]:
# 2761
financial.query('SecuritiesCode == 2761').tail()

In [ ]:
financial.columns

free storage

In [ ]:
del financial

## Options

In [ ]:
options = pd.read_csv('data/train_files/options.csv', parse_dates=[1])

In [ ]:
options.head(2)

In [ ]:
options.columns

will not be used

In [ ]:
options.OptionsCode.nunique()

In [ ]:
del options

## Submission

In [ ]:
submission = pd.read_csv('data/example_test_files/sample_submission.csv', date_parser=[0])
submission.shape

In [ ]:
submission.nunique()

In [ ]:
submission.head()

Target: calculate the rank for each security code for the next 56 working days

In [ ]:
del submission

## Stock prices

In [ ]:
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
prices.shape

In [ ]:
prices.head(2)

### - date

In [ ]:
date = prices.Date
date_desc = date.describe(datetime_is_numeric=True)
date_min = date_desc.loc['min']
date_max = date_desc.loc['max']
print('Data from {:%Y-%m-%d} to {:%Y-%m-%d}'
        .format(date_min, date_max))
print('Data observations {} '.format(
                 date.nunique()))

from https://www.timeanddate.com/date/workdays.html?d1=4&m1=1&y1=2017&d2=3&m2=12&y2=2021&ti=on&

1211 days 

1795 calendar days – 584 days skipped:
Excluded 256 Saturdays
Excluded 256 Sundays
Excluded 72 holidays

excluded 2010-10-1 because system failer in jpx

remain 1210 days  and 8 left
- Which one are missing and why?

1202 == 1794 . we miss data, where?
- find missing data

- generate working days

In [ ]:
# test handel with timeseries in dataframe
t = np.arange(date_min, date_max, timedelta(days=1)).astype(datetime)

#t_work = [day for day in t if asia.Japan().is_working_day(day)]
date_df = pd.DataFrame(date)
#time_df = pd.to_datetime(t_work)
type(pd.Timestamp(date.unique()[0]))

- observations

In [ ]:
prices.info()

In [ ]:
prices.head(2)

In [ ]:
prices.tail(2)

- amount of trades each day

In [ ]:
count = pd.DataFrame(prices.Date.value_counts().reset_index())
count.head()
plt.figure(figsize = (20,5))
sns.lineplot(data = count, x= 'index', y ='Date')


are stocks added during the time? 
compare the stocks in 1 feb 2017 with the stocks in 1 Feb 2121 

In [ ]:
# stock for each days
feb2017 = prices[prices['Date']=='2017-2-1']
feb2021 = prices[prices['Date']=='2021-2-1']

print('SecuritiesCode for Feb 2017 : {} \nSecuritiesCode for Feb 2021 : {}'. format(
                                    feb2017.SecuritiesCode.nunique(), feb2021.SecuritiesCode.nunique()
))

In [ ]:
# new Securites Code in 2021
codes_2017 = feb2017.SecuritiesCode.to_list()
new_codes = feb2021[~feb2021['SecuritiesCode'].isin(codes_2017)]
old_codes = feb2021[~feb2021['SecuritiesCode'].isin(new_codes.SecuritiesCode.to_list())]
print(new_codes.SecuritiesCode.count())
print(old_codes.SecuritiesCode.count())

In [ ]:
def plot_stock(df, Code, feature='Target'):
    df = df.query('SecuritiesCode==@Code')
    #plt.figure(figsize=(20,5))
    sns.lineplot(data=df, y=feature, x='Date', label=code)

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
for code in [1375, 1301]:
    plot_stock(prices, code)

    ax.xaxis.set_major_formatter(DateFormatter("%m-%Y"))

there are 135 Securities codes not listed in 2017. 
may be they were launch later. 

### nan values

In [ ]:
a = prices[(prices['Date']=='2021-12-03')]
a.count()

In [ ]:
a.query('Open != Open')

a day before

In [ ]:
b = prices[(prices['Date']=='2021-12-02')]
b.query('Open != Open')

and before

In [ ]:
c = prices[(prices['Date']=='2021-12-01')]
c.query('Open != Open')

In [ ]:
d = prices[(prices['Date']=='2021-12-01')]
c.query('Open != Open')

In [ ]:

plt.figure(figsize=(25,10))
sns.lineplot(data = prices.query('SecuritiesCode == 2761 and Date > "2021-11-01"' ), x='Date', y= 'Open')

#sns.lineplot(data = prices.query('SecuritiesCode == 9994 and Date > "2021-11-01"' ), x='Date', y= 'Open')

In [ ]:
plt.figure(figsize=(16,10))
sns.lineplot(data = prices.query('SecuritiesCode == 3540'), x='Date', y= 'Open')
sns.lineplot(data = prices.query('SecuritiesCode == 9994'), x='Date', y= 'Open')


In [ ]:
prices.isnull().sum().sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.heatmap(prices.isna(), cbar=False, cmap="plasma",yticklabels=False)
plt.show()

In [ ]:
prices_nan = prices.query('Open != Open')
prices_nan.shape

In [ ]:
prices_nan.Date.value_counts()

In [ ]:
prices_nan[(prices_nan['Date']!='2020-10-01')].Date.value_counts().sort_values(ascending=True)

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(prices_nan.isna(), cbar=False, cmap="plasma",yticklabels=False)
plt.show()

- forward fill for 1-10-2020
- / consecutive missing values of trades can be removed. the stock was taken out
- forward fill for missing values

In [ ]:
prices_wo_nan = prices.query('Date != "2020-10-01" and Open == Open')

In [ ]:
prices_wo_nan.isnull().sum().sort_values(ascending=False)

### SupervisionFlag
Flag of Securities Under Supervision & Securities to Be Delisted
https://www.jpx.co.jp/english/listing/market-alerts/supervision/00-archives/index.html )


If a stock is designated as Securities Under Supervision or Securities to Be Delisted during the private period, it will be excluded from investment after the date of designation

In [ ]:
prices.head()

In [ ]:
prices.SupervisionFlag.value_counts()

In [ ]:
from functions import plot_stock
w_flag = prices[(prices['SupervisionFlag']==True)]
w_flag.shape


In [ ]:
w_flag.head()

In [ ]:
print(f'Securities codes with flag: {w_flag.SecuritiesCode.nunique()} ')
print(f'Securities codes: {w_flag.SecuritiesCode.unique()} : ')

In [ ]:
codes = w_flag.SecuritiesCode.unique()

for code in codes[:3]:
    plot_stock(prices, code)
    plot_stock(w_flag, code)
    plt.show()

In [ ]:
date = w_flag.Date
date_desc = date.describe(datetime_is_numeric=True)
date_min = date_desc.loc['min']
date_max = date_desc.loc['max']
print('Data from {:%Y-%m-%d} to {:%Y-%m-%d}'
        .format(date_min, date_max))
print('Data observations {} '.format(
                 date.nunique()))

the flags are set for the whole period of time

- how long has a stock a flag?

In [ ]:
flag_day = {}
for code in codes: 
    current = w_flag.query('SecuritiesCode == @ code')
    date = current.Date
    date_desc = date.describe(datetime_is_numeric=True)
    date_min = date_desc.loc['min']
    date_max = date_desc.loc['max']
    flag_day[code] = [date_min, date_max, date.nunique()]


In [ ]:
flag_stats = pd.DataFrame(flag_day).T
flag_stats.columns = ['start_flag', 'end_flag', 'days_w_flag']
flag_stats

encode flag: 

In [ ]:
prices.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(prices["SupervisionFlag"])
prices["SupervisionFlag"]=enc.transform(prices["SupervisionFlag"])

In [ ]:
prices.SupervisionFlag.value_counts()

### Target
Change ratio of adjusted closing price between t+2 and t+1 where t+0 is TradeDate <br>
r(k,t) = C(k, t+2) - C(k, t+1) / C(k, t+1)

- adjusted stock price is used

In [ ]:
sns.histplot(prices.Target, bins=800, )
plt.xlim([-0.4, 0.4])

target for a stock

In [ ]:
def plot_target(df, Code):
    single_stock = df.query('SecuritiesCode==@Code')
    plt.figure(figsize=(20,5))
    sns.lineplot(data=single_stock, y='Close', x='Date')
    ax2 = plt.twinx()
    sns.lineplot(data=single_stock, y='Target', x='Date', color='red')  

In [ ]:
for i in [8876, 6630, 2761, 2753]:
    plot_target(prices, i)

- target

#### - Rank
for each day

In [ ]:
prices.Date.nunique()

In [ ]:
prices_rank = pd.DataFrame(columns = prices.columns)

In [ ]:
# only one day
def calculate_rank(df, day):
    df = prices[(prices['Date']== day)]
    df.loc[:, 'Rank'] = df['Target'].rank(ascending=False, method='first') - 1
    
    return df



df = calculate_rank(prices, prices.Date.unique()[0])
one_day_rank = pd.concat([prices_rank, df], axis=0)



In [ ]:
def daily_spread_return(df, day):
    # calculate rank 
    df = prices[(prices['Date']== day)]
    df.loc[:, 'Rank'] = df['Target'].rank(ascending=False, method='first') - 1

    # calculate weights and weighted_target
    weights = np.linspace(start = 2, stop=1, num=200)

    # sum_up
    top200 = df.sort_values('Rank').iloc[:200,:]
    top200['Weights'] = weights
    top200['weigh_Target'] = top200['Target'] * top200['Weights']
    sum_up = top200['weigh_Target'].sum()/top200['Weights'].mean()

    # sum_down
    bottom200 = one_day_rank.sort_values('Rank').iloc[-200:,:].sort_values('Rank', ascending=False)
    bottom200['Weights'] = np.linspace(start = 2, stop=1, num=200)
    bottom200['weigh_Target'] = bottom200['Target'] * bottom200['Weights']
    sum_down = bottom200['weigh_Target'].sum()/bottom200['Weights'].mean()

    # return daily_spread return
    return  df.SecuritiesCode.nunique(), sum_up - sum_down

In [ ]:
stocks_per_day = {}

def calculate_rank(df, day):
    df = prices[(prices['Date']== day)]
    df.loc[:, 'Rank'] = df['Target'].rank(ascending=False, method='first') - 1
    
    return df

prices_rank = pd.DataFrame(columns = prices.columns)

for day in tqdm(prices.Date.unique()):
    stocks, dsr = daily_spread_return(prices, day)
    stocks_per_day[day] = [stocks, dsr]

In [ ]:
stocks_day = pd.DataFrame(stocks_per_day).T
stocks_day.columns = ['SecuritiesCode_s', 'daily_spread_return']
stocks_day

In [ ]:
stocks_day.nunique()

- best and worst ranks are weighted with 1-2  [link](https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition)

## Trades

In [ ]:
trades = pd.read_csv('data/train_files/trades.csv')
trades.shape
trades.head(2)

In [ ]:
trades.shape

only  nan values?

# Features engineering

## 1. fill nan values 

## 2. Adjusted prices
to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares/)

In [ ]:
prices.AdjustmentFactor.unique()

- amount of stocks for each adjustment factor

In [ ]:

for i in sorted(prices.AdjustmentFactor.unique()):
    print( 'adjustment factor = {}   \t:  {} trades'.format(
        round(i,2), prices[(prices['AdjustmentFactor']==i)].Date.count()
    ))
   

In [ ]:
def adjust_price(df):
    # cumulative adjustment factor considering the day shift
    df.sort_values('Date')
    df.loc[:,'CAF'] = df['AdjustmentFactor'].cumprod().shift(1)
    # fill nan values
    df.CAF.fillna(1, inplace=True)
    # prices to be adjusted
    prices =[ 'Open', 'High', 'Low', 'Close']

    for x in prices:
        df.loc[:,'ad_' + str(x)]  = df[x] / df['CAF']
    
    # adjust volume
    df['ad_Volume'] = df['Volume'] * df['CAF']
    df.drop('CAF', axis=1, inplace=True)

     # adjust target
    df.sort_values('Date')
    df['ad_Close_1'] = df['ad_Close'].shift(-1)
    df['ad_Close_2'] = df['ad_Close'].shift(-2)
    df['ad_Target'] = (df['ad_Close_2']-df['ad_Close_1'])/df['ad_Close_1']
    df.drop(['ad_Close_1', 'ad_Close_2'], axis=1, inplace=True)

    return df

In [ ]:
# create an empty new DataFrame
adjusted_data = pd.DataFrame(columns = prices.columns)

for i in tqdm([8876, 6630, 7453, 7638]): #tqdm(prices.SecuritiesCode.unique()):
    df = prices.query('SecuritiesCode ==@i')
    adjusted_df = adjust_price(df)
    adjusted_data = pd.concat([adjusted_data, adjusted_df ], axis=0)

In [ ]:
def plot_stock(df, Code):
    single_stock = df.query('SecuritiesCode==@Code')
    plt.figure(figsize=(20,5))
    sns.lineplot(data=single_stock, y='Close', x='Date')
    ax2 = plt.twinx()
    sns.lineplot(data=single_stock, y='ad_Close', x='Date', color='red')


In [ ]:
for i in [8876, 6630, 7453, 7638]:
    plot_stock(adjusted_data, i)

data/train_files/stock_prices_ad.csvthe adjusted value was calculated with process_adjusted_price.py and save in : 

In [ ]:
adjusted_data = pd.read_csv('data/train_files/stock_prices_ad.csv', parse_dates=[2], index_col=0)
adjusted_data.head(2)

In [ ]:
def plot_stock(df, Code):
    df = df.query('SecuritiesCode==@Code')
    plt.figure(figsize=(20,5))
    sns.lineplot(data=df, y='Target', x='Date')
    sns.lineplot(data=df, y='ad_Target', x='Date', color = 'red')
    ax2 = plt.twinx()
    sns.lineplot(data=df, y='Close', x='Date')

for i in [8876, 6630, 7453, 7638]:
    plot_stock(new_df, i)

## 5. Close, Open, High and Low price lag 1


In [ ]:
df.columns

In [ ]:


features = ['ad_Close', 'ad_Open', 'ad_High' , 'ad_Low', 'ad_Volume']
new_features = [x + "_lag1" for x in features]

feat_data = pd.DataFrame(columns = adjusted_data.columns)

def feature_lag(df, features, lag=1):
    for feat in features:
        df[name] = df[str(feat)].shift(lag)
        #new_features.append(name)
    return df


df = adjusted_data.query('SecuritiesCode ==@i')
df = feature_lag(df, features)



In [ ]:
df[new_features + features].head()

## 3. RSI

In [ ]:
def rsi (df_serie,periods = 14, ema = True): 
    """Relative Strength Index
        RSI = 100–(100/1 + RS) ; RS = Av Gain / Av loss
    Args:
        df_serie (_type_): series of ad_Close
        periods (int, optional): 5,7,9,14,21,30 Defaults to 14.
        ema (bool, optional): _description_. Defaults to True.

    Returns:
        pd.Series: _description_
    """
        
    close_delta = df_serie.diff() # .dropna()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    if ema == True: # exponential moving average
        ma_up = up.ewm(com= periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    else: # using moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.ewm(window = periods, adjust=False).mean()
        
    rs = ma_up / ma_down
    rsi = 100 - (100 / (1+rs))

    return rsi

In [ ]:
from functions import df_security_code

df = df_security_code(adjusted_data,8194 )
df['RSI'] = rsi(df['ad_Close'])

In [ ]:
from functions import plot_stock

plt.figure(figsize=(20,5))
plot_stock(df, 8194 ,'RSI')
ax2 = plt.twinx()
plot_stock(df, 8194 ,'ad_Close', color='red')

## 4. Return

In [ ]:
df['Return'] = df['ad_Close'].pct_change()
# adjusted_data['Return_m'] = adjusted_data['ad_Close'].resample('M').ffill().pct_change()


In [ ]:
plot_stock(df, 8194, 'Return')

In [ ]:
# cumulative
df['cum_Return'] = (df['Return'] + 1).cumprod()
plot_stock(df, 8194, 'cum_Return')

## 5. Simple moving average





In [ ]:
# ussualy period 10 or 5
# setting for all prices



features = ['ad_Close', 'ad_Open', 'ad_High' , 'ad_Low', 'ad_Volume']
new_features = [x + "_sma10" for x in features]

feat_data = pd.DataFrame(columns = adjusted_data.columns)

def SMA(df, features, period=10):
    for feat in features:
        name = feat + "_sma" + str(period)
        df[name] = df[feat].rolling(window=period).mean()  # only for close or for all values?
        #new_features.append(name)
    return df


df = adjusted_data.query('SecuritiesCode == 8194')
df = SMA(df, features)

In [ ]:
df[features +  new_features].head(20)

## 6. MACD : The Moving Average Convergence Divergence 



In [ ]:

def macd(df):
    ema26 = df['ad_Close'].ewm(span=26, adjust=False, min_periods=26).mean()
    ema12 = df['ad_Close'].ewm(span=12, adjust=False, min_periods=12).mean()

    macd = ema12 - ema26

    # Get the 9-Day EMA of the MACD for the Trigger line

    macd_ema9 = macd.ewm(span=9, adjust=False, min_periods=9).mean()

    # calculate the difference
    macd_diff = macd - macd_ema9

    df['macd'] = df.index.map(macd)
    df['macd_h'] = df.index.map(macd_diff)
    df['macd_s'] = df.index.map(macd_ema9)

    return df

In [ ]:
df = adjusted_data.query('SecuritiesCode == 8194')
df = macd(df)

In [ ]:
df.tail()

In [ ]:

plot_stock(df, 8194, 'ad_Close', color='red')
plot_stock(df, 8194, 'macd')

## 7. 